In [5]:
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import numpy as np
import networkx as nx
import datetime as dt
import random
import pickle
import math
import csv
import os

%matplotlib inline

### Visualizations & plots

In [9]:
home_dir = os.path.expanduser("~")
code_dir = os.path.join(home_dir,"Developer","GitHub","endogenous-growth")
nets_dir = os.path.join(code_dir,"networks")
figs_dir = os.path.join(code_dir,"tmp")

In [ ]:
from src.models import InOneOutOne, InOneOutTwo, InTwoOutOne, InOutThree, InTwoOutTwo

### Node dynamics -- node trajectories

In [8]:
# DEFINE NETWORKS
runs = list(range(50))
gammas = [0,4,5,6,8]

m = 3
alpha = 0.95
N = 100

i_s = [3,10,20,30,40,50,60,70,80,90] # nodes entering at step i

# NORMALIZED RANK OF NODES i
ranks = {}
ranks_avg = {}
scores = {}
scores_avg = {}

for gamma in gammas:

    # STORE OUTPUT
    ranks[gamma] = {}
    ranks_avg[gamma] = {} 
    scores[gamma] = {}
    scores_avg[gamma] = {}

    for i in i_s:

        # STORE OUTPUT
        ranks[gamma][i] = [] 
        scores[gamma][i] = [] 
        
        for run in runs:
            # DEFINE NETWORK
            network = "_".join(["i1o1","m"+str(m),"a"+str(alpha),"g"+str(gamma)])

            # LOAD RUN
            with open(os.path.join(code_dir,"networks",network,"run_"+str(run)+'.pkl'), 'rb') as f:
                model = pickle.load(f)

            # UNPACK
            networks = model.networks
            
            # IDENTIFY NODE i
            n_i = max(networks[i-1].nodes())

            # GET SCORES
            score = [G.nodes[n_i]['score'] for G in networks[i-1:]]
            scores[gamma][i].append(np.array(score))

            # GET RANKS
            valuess = [nx.get_node_attributes(G, 'score') for G in networks[i-1:]]
            rankings = [{node: rank for rank, node in enumerate(sorted(values, key=values.get))} for values in valuess]
            rank = [ranking[n_i]/(i+j-1) for j, ranking in enumerate(rankings)]
            ranks[gamma][i].append(np.array(rank))
                
        # GET AVERAGE RANKS
        ranks_avg[gamma][i] = np.zeros(len(ranks[gamma][i][0]))
        for run in runs:
            ranks_avg[gamma][i] = ranks_avg[gamma][i] + ranks[gamma][i][run]
        ranks_avg[gamma][i] = ranks_avg[gamma][i]/len(runs)

        # GET AVERAGE SCORES
        scores_avg[gamma][i] = np.zeros(len(scores[gamma][i][0]))
        for run in runs:
            scores_avg[gamma][i] = scores_avg[gamma][i] + scores[gamma][i][run]
        scores_avg[gamma][i] = scores_avg[gamma][i]/len(runs)

    # READY OUTPUT DIRECTORY
    output = "_".join(["i1o1","m"+str(m),"a"+str(alpha)])
    if not os.path.exists(os.path.join(figs_dir,output)):
        os.mkdir(os.path.join(figs_dir,output))

In [11]:
# GAMMAS
plot_gammas = [0,4,8]
N=100

# Create a colorbar
cmap = plt.colormaps['BuPu'] 
colors = [cmap(i) for i in np.linspace(0.2, 1, len(i_s))]

#initialize figure
fig, ax = plt.subplots(1, 3, dpi=200, figsize=(12,4), sharex=True, sharey=True)

for c, i in enumerate(i_s):

       x = range(i,N+1)

       # Plot the instances
       for run in runs:
              ax[0].plot(x, ranks[plot_gammas[0]][i][run], linestyle='-', linewidth = 0.5, alpha=0.1, color=colors[c])
              ax[1].plot(x, ranks[plot_gammas[1]][i][run], linestyle='-', linewidth = 0.5, alpha=0.1, color=colors[c])
              ax[2].plot(x, ranks[plot_gammas[2]][i][run], linestyle='-', linewidth = 0.5, alpha=0.1, color=colors[c])

       # Plot the averages
       ax[0].plot(x, ranks_avg[plot_gammas[0]][i], linestyle='-', linewidth = 2, alpha=1, color=colors[c])
       ax[1].plot(x, ranks_avg[plot_gammas[1]][i], linestyle='-', linewidth = 2, alpha=1, color=colors[c])
       ax[2].plot(x, ranks_avg[plot_gammas[2]][i], linestyle='-', linewidth = 2, alpha=1, color=colors[c])

ax[0].set_xticks([3]+list(range(20,N+1,20)))
ax[0].set_xticklabels(['N=3', '20', '40', '60', '80', '100'])
# Remove the first tick mark
#for i in range(3):
#       lines = ax[i].get_xticklines()
#       lines[0].set_visible(False)

ax[0].set_xlim(0,N)
ax[0].set_ylim(0,1)

ax[0].set_title("Random ("+r'$\gamma$'+f' = {plot_gammas[0]})', fontsize=14)
ax[1].set_title("Opportunistic ("+r'$\gamma$'+f' = {plot_gammas[1]})', fontsize=14)
ax[2].set_title("Near-optimal ("+r'$\gamma$'+f' = {plot_gammas[2]})', fontsize=14)
ax[0].set_ylabel("Relative rank", fontsize=12)


plt.savefig(os.path.join(figs_dir,output,"node-dynamics.jpeg"), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(figs_dir,output,"node-dynamics.pdf"), bbox_inches='tight')
plt.close()

## Attractiveness

In [ ]:
# DISTRIBUTIONS
runs = list(range(44))
gammas = [0,1,2,3,4,5,6,7,8,9,"inf"]

PRs = {}
for gamma in gammas:
    PRs[gamma] = []
    for run in runs:    
        network = "_".join(["i1o1","m"+str(m),"a"+str(alpha),"g"+str(gamma)])

        # LOAD RUN
        try:
            with open(os.path.join(nets_dir,network,"run_"+str(run)+'.pkl'), 'rb') as f:
                model = pickle.load(f)
        except:
            continue

        # Get the PageRanks of the nodes at step 100
        PR = nx.get_node_attributes(model.networks[-1], "score")

        # Save it
        PRs[gamma].append(PR)

In [ ]:
plot_gammas = [0,2,4,8,"inf"]

# Create a colorbar
cmap = plt.colormaps['BuPu'] 
colors = [cmap(i) for i in np.linspace(0.333, 1, len(plot_gammas))]

# Create the figure
fig, ax = plt.subplots()

# Plot the scatterplots
for i, gamma in enumerate(plot_gammas):
    color = colors[i]
    for PR in PRs[gamma]:
        # get the rank of the nodes, starting from one
        PR_ranksort = sorted(PR.items(), key=lambda x: x[1], reverse=True)
        ranks = {node: rank for rank, (node, score) in enumerate(PR_ranksort, start=1)}
        # get the entry time of the nodes, starting from one
        PR_nodesort = sorted(PR.items(), key=lambda x: x[0])        
        entry = {node: entry for entry, (node, score) in enumerate(PR_nodesort, start=1)}
        # turn these into lists
        x_pts = [entry[node] for node in PR]
        y_pts = [ranks[node] for node in PR]
        # plot scatterplots
        plot = ax.scatter(x_pts, y_pts, color = color, lw = 0.5, alpha = 0.2)

# Plot the overall distributions
#for i, gamma in enumerate(plot_gammas):
#    color = colors[i]    
#    # get combined PDF
#    x_pdf, y_pdf = get_distribution([item for sublist in Vs[gamma] for item in sublist], number_of_bins = 11)
#    # plot combined PDF
#    label = f'$\\gamma = ${gamma}' if gamma != "inf" else f'$\\gamma \\to \\infty$'
#    plot = plt.loglog(x_pdf, y_pdf, color = color, marker = "s",
#                    lw = 1.5, label=label)

# labels
ax.set_xlabel("Entry Step", fontsize = 14)
ax.set_ylabel("Rank", fontsize = 14)
ax.invert_xaxis()
ax.invert_yaxis()
#plt.legend(loc='upper right',frameon=False, fontsize=14)
# save
plt.savefig(os.path.join(figs_dir,output,"node-dynamics.jpeg"))
plt.savefig(os.path.join(figs_dir,output,"node-dynamics.pdf"))
plt.tight_layout()
plt.show()